# Clickbait detection

## Normalización del dataset combinando los aspectos mencionados en las especificaciones

In [1]:
# Añadir el directorio 'src' al sys.path
import sys
import os

In [2]:
# Obtener la ruta absoluta de la carpeta 'src' para que sea accesible desde el notebook
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

In [3]:
# Importar la función de normalización
from preprocessing import normalize_text

In [4]:
# Importar pandas para cargar y manipular el dataset
import pandas as pd

In [5]:
# Cargar el dataset
df = pd.read_csv("../data/TA1C_dataset_detection_train.csv")


In [6]:
df = df.drop(['Tweet Date', 'Media Name', 'Media Origin'], axis=1)
# Aplicar cada tipo de normalización
df['tokenized_text'] = df['Teaser Text'].apply(lambda x: normalize_text(x, mode="tokenization"))
df['cleaned_text'] = df['Teaser Text'].apply(lambda x: normalize_text(x, mode="text_cleaning"))
df['no_stopwords_text'] = df['Teaser Text'].apply(lambda x: normalize_text(x, mode="remove_stopwords"))
df['lemmatized_text'] = df['Teaser Text'].apply(lambda x: normalize_text(x, mode="lemmatization"))
df['tokenized_cleaned_text'] = df['Teaser Text'].apply(lambda x: normalize_text(x, mode="text_cleaning")).apply(lambda x: normalize_text(x, mode="tokenization"))
df["tokenized_cleaned_text_no_stopwords"] = df['Teaser Text'].apply(lambda x: normalize_text(x, mode="text_cleaning")).apply(lambda x: normalize_text(x, mode="remove_stopwords"))
df["tokenized_cleaned_text_no_stopwords_lemmatized"] = df['tokenized_cleaned_text_no_stopwords'].apply(lambda x: normalize_text(" ".join(x), mode='lemmatization'))


In [7]:
# Mostrar ejemplos de cada tipo de normalización
print("\nEjemplos de normalización:")
print(df[['Teaser Text', 'tokenized_text', 'cleaned_text', 'no_stopwords_text', 'lemmatized_text', 'tokenized_cleaned_text', 'tokenized_cleaned_text_no_stopwords', 'tokenized_cleaned_text_no_stopwords_lemmatized']].head())



Ejemplos de normalización:
                                         Teaser Text  \
0  #SegundaDivisión  | La fortaleza del ataque: R...   
1  Jorge Lanata a los argentinos que se van a Uru...   
2  Raffo: “Los montevideanos deben estar alerta p...   
3  Ecos del universo: joven uruguayo desentraña (...   
4  Propuesta quinquenal de ANEP: aumento de 3,8% ...   

                                      tokenized_text  \
0  [#, SegundaDivisión,  , |, La, fortaleza, del,...   
1  [Jorge, Lanata, a, los, argentinos, que, se, v...   
2  [Raffo, :, “, Los, montevideanos, deben, estar...   
3  [Ecos, del, universo, :, joven, uruguayo, dese...   
4  [Propuesta, quinquenal, de, ANEP, :, aumento, ...   

                                        cleaned_text  \
0  segundadivisión   la fortaleza del ataque ramp...   
1  jorge lanata a los argentinos que se van a uru...   
2  raffo “los montevideanos deben estar alerta po...   
3  ecos del universo joven uruguayo desentraña y ...   
4  propuesta quinq

In [8]:

# Guardar el dataset con todas las columnas de normalización
output_path = "../data/TA1C_dataset_detection_train_cleaned.csv"
df.to_csv(output_path, index=False)
print(f"Dataset con normalizaciones guardado en '{output_path}'")

Dataset con normalizaciones guardado en '../data/TA1C_dataset_detection_train_cleaned.csv'


### Dividimos el dataset dentro del Notebook

In [11]:
from sklearn.model_selection import train_test_split

# Dividir el dataset en conjuntos de entrenamiento y desarrollo
train_df, dev_df = train_test_split(
    df,
    test_size=0.25,  # 25% para desarrollo
    random_state=0,  # Para reproducibilidad
    stratify=df['Tag Value'],  # Estratificar según la columna de clase
    shuffle=True
)

# Guardar los conjuntos en archivos CSV
train_df.to_csv("../data/TA1C_dataset_detection_train_split.csv", index=False)
dev_df.to_csv("../data/TA1C_dataset_detection_dev_split.csv", index=False)

print("Conjuntos de entrenamiento y desarrollo guardados en '../data/'")

Conjuntos de entrenamiento y desarrollo guardados en '../data/'


### Representación de texto

In [12]:
# Cargar los conjuntos de entrenamiento y desarrollo
train_df = pd.read_csv("../data/TA1C_dataset_detection_train_split.csv")
dev_df = pd.read_csv("../data/TA1C_dataset_detection_dev_split.csv")

# Verificar las columnas disponibles
print(train_df.columns)

Index(['Tweet ID', 'Teaser Text', 'Tag Value', 'tokenized_text',
       'cleaned_text', 'no_stopwords_text', 'lemmatized_text',
       'tokenized_cleaned_text', 'tokenized_cleaned_text_no_stopwords',
       'tokenized_cleaned_text_no_stopwords_lemmatized'],
      dtype='object')


## Usando tokenización

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Probar diferentes configuraciones de n-gramas y representaciones
configurations = [
    {"ngram_range": (1, 1), "representation": "tfidf"},  # Unigramas con TF-IDF
    {"ngram_range": (1, 2), "representation": "tfidf"},  # Unigramas + Bigramas con TF-IDF
    {"ngram_range": (1, 3), "representation": "tfidf"},  # Unigramas + Bigramas + Trigramas con TF-IDF
    {"ngram_range": (1, 2), "representation": "binary"},  # Unigramas + Bigramas con representación binaria
    {"ngram_range": (1, 2), "representation": "frequency"},  # Unigramas + Bigramas con frecuencia
]

for config in configurations:
    print(f"Configuración: ngram_range={config['ngram_range']}, representación={config['representation']}")
    
    # Configurar el vectorizador
    if config["representation"] == "tfidf":
        vectorizer = TfidfVectorizer(ngram_range=config["ngram_range"])
    elif config["representation"] == "binary":
        vectorizer = TfidfVectorizer(ngram_range=config["ngram_range"], binary=True)
    elif config["representation"] == "frequency":
        vectorizer = TfidfVectorizer(ngram_range=config["ngram_range"], use_idf=False)
    
    # Crear representaciones para el conjunto de entrenamiento y desarrollo
    X_train = vectorizer.fit_transform(train_df['tokenized_text'])
    X_dev = vectorizer.transform(dev_df['tokenized_text'])
    
    # Verificar las dimensiones
    print(f"Dimensiones de X_train: {X_train.shape}")
    print(f"Dimensiones de X_dev: {X_dev.shape}")

Configuración: ngram_range=(1, 1), representación=tfidf
Dimensiones de X_train: (2100, 11434)
Dimensiones de X_dev: (700, 11434)
Configuración: ngram_range=(1, 2), representación=tfidf
Dimensiones de X_train: (2100, 44247)
Dimensiones de X_dev: (700, 44247)
Configuración: ngram_range=(1, 3), representación=tfidf
Dimensiones de X_train: (2100, 85357)
Dimensiones de X_dev: (700, 85357)
Configuración: ngram_range=(1, 2), representación=binary
Dimensiones de X_train: (2100, 44247)
Dimensiones de X_dev: (700, 44247)
Configuración: ngram_range=(1, 2), representación=frequency
Dimensiones de X_train: (2100, 44247)
Dimensiones de X_dev: (700, 44247)


In [14]:
from sklearn.decomposition import TruncatedSVD

# Reducir dimensionalidad con TruncatedSVD
svd = TruncatedSVD(n_components=100, random_state=0)  # Reducir a 100 dimensiones
X_train_reduced = svd.fit_transform(X_train)
X_dev_reduced = svd.transform(X_dev)

# Verificar las dimensiones después de la reducción
print(f"Dimensiones reducidas de X_train: {X_train_reduced.shape}")
print(f"Dimensiones reducidas de X_dev: {X_dev_reduced.shape}")

Dimensiones reducidas de X_train: (2100, 100)
Dimensiones reducidas de X_dev: (700, 100)


In [18]:
# Convertir etiquetas de texto a valores numéricos
label_mapping = {'Clickbait': 1, 'No': 0}
train_df['Tag Value'] = train_df['Tag Value'].map(label_mapping)
dev_df['Tag Value'] = dev_df['Tag Value'].map(label_mapping)

### Logistic Regression y Validación Cruzada

In [19]:
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer, f1_score

# Configurar validación cruzada estratificada
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

# Definir el modelo
model = LogisticRegression(max_iter=200)

# Evaluar el modelo con validación cruzada usando f1_macro
f1_macro_scorer = make_scorer(f1_score, average='macro')
scores = cross_val_score(model, X_train_reduced, train_df['Tag Value'], cv=cv, scoring=f1_macro_scorer)

# Imprimir los resultados
print(f"F1-macro scores por fold: {scores}")
print(f"F1-macro promedio: {scores.mean():.4f}")

ValueError: Input y contains NaN.

## Usando lematización

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Probar diferentes configuraciones de n-gramas y representaciones
configurations = [
    {"ngram_range": (1, 1), "representation": "tfidf"},  # Unigramas con TF-IDF
    {"ngram_range": (1, 2), "representation": "tfidf"},  # Unigramas + Bigramas con TF-IDF
    {"ngram_range": (1, 3), "representation": "tfidf"},  # Unigramas + Bigramas + Trigramas con TF-IDF
    {"ngram_range": (1, 2), "representation": "binary"},  # Unigramas + Bigramas con representación binaria
    {"ngram_range": (1, 2), "representation": "frequency"},  # Unigramas + Bigramas con frecuencia
]

for config in configurations:
    print(f"Configuración: ngram_range={config['ngram_range']}, representación={config['representation']}")
    
    # Configurar el vectorizador
    if config["representation"] == "tfidf":
        vectorizer = TfidfVectorizer(ngram_range=config["ngram_range"])
    elif config["representation"] == "binary":
        vectorizer = TfidfVectorizer(ngram_range=config["ngram_range"], binary=True)
    elif config["representation"] == "frequency":
        vectorizer = TfidfVectorizer(ngram_range=config["ngram_range"], use_idf=False)
    
    # Crear representaciones para el conjunto de entrenamiento y desarrollo
    X_train = vectorizer.fit_transform(train_df['lemmatized_text'])
    X_dev = vectorizer.transform(dev_df['lemmatized_text'])
    
    # Verificar las dimensiones
    print(f"Dimensiones de X_train: {X_train.shape}")
    print(f"Dimensiones de X_dev: {X_dev.shape}")

Configuración: ngram_range=(1, 1), representación=tfidf
Dimensiones de X_train: (2100, 9183)
Dimensiones de X_dev: (700, 9183)
Configuración: ngram_range=(1, 2), representación=tfidf
Dimensiones de X_train: (2100, 39506)
Dimensiones de X_dev: (700, 39506)
Configuración: ngram_range=(1, 3), representación=tfidf
Dimensiones de X_train: (2100, 79900)
Dimensiones de X_dev: (700, 79900)
Configuración: ngram_range=(1, 2), representación=binary
Dimensiones de X_train: (2100, 39506)
Dimensiones de X_dev: (700, 39506)
Configuración: ngram_range=(1, 2), representación=frequency
Dimensiones de X_train: (2100, 39506)
Dimensiones de X_dev: (700, 39506)


In [14]:
from sklearn.decomposition import TruncatedSVD

# Reducir dimensionalidad con TruncatedSVD
svd = TruncatedSVD(n_components=100, random_state=0)  # Reducir a 100 dimensiones
X_train_reduced = svd.fit_transform(X_train)
X_dev_reduced = svd.transform(X_dev)

# Verificar las dimensiones después de la reducción
print(f"Dimensiones reducidas de X_train: {X_train_reduced.shape}")
print(f"Dimensiones reducidas de X_dev: {X_dev_reduced.shape}")

Dimensiones reducidas de X_train: (2100, 100)
Dimensiones reducidas de X_dev: (700, 100)


In [15]:
# Convertir etiquetas de texto a valores numéricos
label_mapping = {'Clickbait': 1, 'No': 0}
train_df['Tag Value'] = train_df['Tag Value'].map(label_mapping)
dev_df['Tag Value'] = dev_df['Tag Value'].map(label_mapping)

### Logistic Regression y Validación Cruzada

In [16]:
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer, f1_score

# Configurar validación cruzada estratificada
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

# Definir el modelo
model = LogisticRegression(max_iter=200)

# Evaluar el modelo con validación cruzada usando f1_macro
f1_macro_scorer = make_scorer(f1_score, average='macro')
scores = cross_val_score(model, X_train_reduced, train_df['Tag Value'], cv=cv, scoring=f1_macro_scorer)

# Imprimir los resultados
print(f"F1-macro scores por fold: {scores}")
print(f"F1-macro promedio: {scores.mean():.4f}")

F1-macro scores por fold: [0.57350414 0.57255747 0.61648997 0.60008246 0.55428571]
F1-macro promedio: 0.5834


### Entrenamiento del modelo utilizando algoritmos de ML

#### Naive Bayes Multinominal

In [27]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score

# Definir el modelo de Naïve Bayes Multinomial
nb_model = MultinomialNB()

# Evaluar el modelo con validación cruzada usando las representaciones originales (X_train)
nb_scores = cross_val_score(nb_model, X_train, train_df['Tag Value'], cv=5, scoring='f1_macro')

# Imprimir los resultados
print("Naïve Bayes Multinomial (sin TruncatedSVD):")
print(f"F1-macro scores por fold: {nb_scores}")
print(f"F1-macro promedio: {nb_scores.mean():.4f}")

Naïve Bayes Multinomial (sin TruncatedSVD):
F1-macro scores por fold: [0.41666667 0.41666667 0.42551064 0.41747573 0.42638889]
F1-macro promedio: 0.4205


#### Support Vector Machines

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, f1_score, make_scorer

# Definir el modelo y los hiperparámetros a probar
svc = SVC(random_state=0)
param_grid = {
    'kernel': ['linear', 'rbf'],
    'C': [0.1, 1, 10],
    'gamma': ['scale', 'auto']
}

# Configurar validación cruzada y búsqueda de hiperparámetros
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
grid_search = GridSearchCV(
    svc,
    param_grid,
    scoring=make_scorer(f1_score, average='macro'),
    cv=cv,
    n_jobs=-1,
    verbose=2
)

# Ejecutar búsqueda
grid_search.fit(X_train_reduced, train_df['Tag Value'])

# Imprimir mejores resultados
print("Mejor configuración encontrada:")
print(grid_search.best_params_)
print(f"Mejor F1-macro: {grid_search.best_score_:.4f}")

print("\nResultados detallados por configuración:")
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
params = grid_search.cv_results_['params']
for mean, std, param in zip(means, stds, params):
    print(f"{param} -> F1-macro: {mean:.4f} (+/-{std:.4f})")

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time=   0.1s
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time=   0.1s
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time=   0.2s
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time=   0.1s
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time=   0.1s
[CV] END .....................C=0.1, gamma=scale, kernel=rbf; total time=   0.2s
[CV] END .....................C=0.1, gamma=scale, kernel=rbf; total time=   0.3s
[CV] END .....................C=0.1, gamma=scale, kernel=rbf; total time=   0.3s
[CV] END ...................C=0.1, gamma=auto, kernel=linear; total time=   0.1s
[CV] END .....................C=0.1, gamma=scale, kernel=rbf; total time=   0.5s
[CV] END ...................C=0.1, gamma=auto, kernel=linear; total time=   0.1s
[CV] END ...................C=0.1, gamma=auto, k

In [29]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

# Definir el modelo de SVC
svc_model = SVC(C=10,gamma='scale',kernel='linear', random_state=0)

# Evaluar el modelo con validación cruzada usando las representaciones reducidas (X_train_reduced)
svc_scores = cross_val_score(svc_model, X_train_reduced, train_df['Tag Value'], cv=5, scoring='f1_macro')

# Imprimir los resultados
print("Support Vector Machines (SVC):")
print(f"F1-macro scores por fold: {svc_scores}")
print(f"F1-macro promedio: {svc_scores.mean():.4f}")

Support Vector Machines (SVC):
F1-macro scores por fold: [0.72859226 0.67157895 0.71005814 0.68562874 0.71975417]
F1-macro promedio: 0.7031


#### Multi-layer Perceptron (MLPClassifier)

In [18]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import f1_score, make_scorer

# Definir el modelo y los hiperparámetros a probar
mlp = MLPClassifier(random_state=0, max_iter=2000)
param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (100, 50), (150,)],
    'alpha': [0.0001, 0.001, 0.01],
    'solver': ['adam', 'lbfgs'],
}

# Configurar validación cruzada y búsqueda de hiperparámetros
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
grid_search = GridSearchCV(
    mlp,
    param_grid,
    scoring=make_scorer(f1_score, average='macro'),
    cv=cv,
    n_jobs=-1,
    verbose=2
)

# Ejecutar búsqueda
grid_search.fit(X_train_reduced, train_df['Tag Value'])

# Imprimir mejores resultados
print("Mejor configuración encontrada:")
print(grid_search.best_params_)
print(f"Mejor F1-macro: {grid_search.best_score_:.4f}")

print("\nResultados detallados por configuración:")
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
params = grid_search.cv_results_['params']
for mean, std, param in zip(means, stds, params):
    print(f"{param} -> F1-macro: {mean:.4f} (+/-{std:.4f})")

Fitting 5 folds for each of 24 candidates, totalling 120 fits


[CV] END alpha=0.0001, hidden_layer_sizes=(50,), solver=adam; total time=  12.0s
[CV] END alpha=0.0001, hidden_layer_sizes=(50,), solver=adam; total time=  12.8s
[CV] END alpha=0.0001, hidden_layer_sizes=(50,), solver=adam; total time=  13.8s
[CV] END alpha=0.0001, hidden_layer_sizes=(50,), solver=adam; total time=  14.8s
[CV] END alpha=0.0001, hidden_layer_sizes=(50,), solver=lbfgs; total time=   1.5s
[CV] END alpha=0.0001, hidden_layer_sizes=(50,), solver=lbfgs; total time=   4.5s
[CV] END alpha=0.0001, hidden_layer_sizes=(50,), solver=lbfgs; total time=   2.8s
[CV] END alpha=0.0001, hidden_layer_sizes=(50,), solver=lbfgs; total time=   3.5s
[CV] END alpha=0.0001, hidden_layer_sizes=(50,), solver=lbfgs; total time=   3.6s
[CV] END alpha=0.0001, hidden_layer_sizes=(50,), solver=adam; total time=  15.1s
[CV] END alpha=0.0001, hidden_layer_sizes=(100,), solver=adam; total time=  15.1s
[CV] END alpha=0.0001, hidden_layer_sizes=(100,), solver=adam; total time=  15.9s
[CV] END alpha=0.0001

/home/omar/Documentos/ESCOM/NLP/ClickbaitDetection/venv/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/home/omar/Documentos/ESCOM/NLP/ClickbaitDetection/venv/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


[CV] END .alpha=0.01, hidden_layer_sizes=(50,), solver=lbfgs; total time=  14.3s
[CV] END .alpha=0.01, hidden_layer_sizes=(50,), solver=lbfgs; total time=  13.6s
[CV] END ..alpha=0.01, hidden_layer_sizes=(50,), solver=adam; total time=  17.5s


/home/omar/Documentos/ESCOM/NLP/ClickbaitDetection/venv/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


...

[CV] END alpha=0.01, hidden_layer_sizes=(150,), solver=lbfgs; total time=  43.7s


/home/omar/Documentos/ESCOM/NLP/ClickbaitDetection/venv/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


[CV] END alpha=0.01, hidden_layer_sizes=(150,), solver=lbfgs; total time=  45.9s


/home/omar/Documentos/ESCOM/NLP/ClickbaitDetection/venv/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


[CV] END alpha=0.01, hidden_layer_sizes=(150,), solver=lbfgs; total time=  34.9s
Mejor configuración encontrada:
{'alpha': 0.01, 'hidden_layer_sizes': (100,), 'solver': 'adam'}
Mejor F1-macro: 0.7230

Resultados detallados por configuración:
{'alpha': 0.0001, 'hidden_layer_sizes': (50,), 'solver': 'adam'} -> F1-macro: 0.7111 (+/-0.0136)
{'alpha': 0.0001, 'hidden_layer_sizes': (50,), 'solver': 'lbfgs'} -> F1-macro: 0.6870 (+/-0.0260)
{'alpha': 0.0001, 'hidden_layer_sizes': (100,), 'solver': 'adam'} -> F1-macro: 0.7215 (+/-0.0200)
{'alpha': 0.0001, 'hidden_layer_sizes': (100,), 'solver': 'lbfgs'} -> F1-macro: 0.6871 (+/-0.0197)
{'alpha': 0.0001, 'hidden_layer_sizes': (100, 50), 'solver': 'adam'} -> F1-macro: 0.6975 (+/-0.0269)
{'alpha': 0.0001, 'hidden_layer_sizes': (100, 50), 'solver': 'lbfgs'} -> F1-macro: 0.6783 (+/-0.0189)
{'alpha': 0.0001, 'hidden_layer_sizes': (150,), 'solver': 'adam'} -> F1-macro: 0.7147 (+/-0.0264)
{'alpha': 0.0001, 'hidden_layer_sizes': (150,), 'solver': 'lbfgs'

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score

# Definir el modelo de MLP
mlp_model = MLPClassifier(hidden_layer_sizes=(100,), max_iter=2000, alpha=0.01, solver='adam', random_state=0)

# Evaluar el modelo con validación cruzada usando las representaciones reducidas (X_train_reduced)
mlp_scores = cross_val_score(mlp_model, X_train_reduced, train_df['Tag Value'], cv=5, scoring='f1_macro')

# Imprimir los resultados
print("Multi-layer Perceptron (MLPClassifier):")
print(f"F1-macro scores por fold: {mlp_scores}")
print(f"F1-macro promedio: {mlp_scores.mean():.4f}")

/home/omar/Documentos/ESCOM/NLP/ClickbaitDetection/venv/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:698: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
/home/omar/Documentos/ESCOM/NLP/ClickbaitDetection/venv/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:698: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


#### Random Forest

In [32]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import f1_score, make_scorer

# Definir el modelo y los hiperparámetros a probar
rf = RandomForestClassifier(random_state=0)
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Configurar validación cruzada y búsqueda de hiperparámetros
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
grid_search = GridSearchCV(
    rf,
    param_grid,
    scoring=make_scorer(f1_score, average='macro'),
    cv=cv,
    n_jobs=-1,
    verbose=2
)

# Ejecutar búsqueda
grid_search.fit(X_train_reduced, train_df['Tag Value'])

# Imprimir mejores resultados
print("Mejor configuración encontrada:")
print(grid_search.best_params_)
print(f"Mejor F1-macro: {grid_search.best_score_:.4f}")

print("\nResultados detallados por configuración:")
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
params = grid_search.cv_results_['params']
for mean, std, param in zip(means, stds, params):
    print(f"{param} -> F1-macro: {mean:.4f} (+/-{std:.4f})")

Fitting 5 folds for each of 108 candidates, totalling 540 fits
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.9s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   1.0s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   1.0s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   1.0s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   1.0s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   1.9s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   2.1s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   2.3s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   1.9s
[CV] END max_de

In [33]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Definir el modelo de Random Forest
rf_model = RandomForestClassifier(max_depth=30, min_samples_leaf=1, min_samples_split=10,n_estimators=50, random_state=0)

# Evaluar el modelo con validación cruzada usando las representaciones reducidas (X_train_reduced)
rf_scores = cross_val_score(rf_model, X_train_reduced, train_df['Tag Value'], cv=5, scoring='f1_macro')

# Imprimir los resultados
print("Random Forest:")
print(f"F1-macro scores por fold: {rf_scores}")
print(f"F1-macro promedio: {rf_scores.mean():.4f}")

Random Forest:
F1-macro scores por fold: [0.66965786 0.60441819 0.63211679 0.64269706 0.64793972]
F1-macro promedio: 0.6394


#### Gradient Boosting

In [34]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import f1_score, make_scorer

# Definir el modelo y los hiperparámetros a probar
gb = GradientBoostingClassifier(random_state=0)
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [2, 3, 5],
    'subsample': [0.8, 1.0]
}

# Configurar validación cruzada y búsqueda de hiperparámetros
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
grid_search = GridSearchCV(
    gb,
    param_grid,
    scoring=make_scorer(f1_score, average='macro'),
    cv=cv,
    n_jobs=-1,
    verbose=2
)

# Ejecutar búsqueda
grid_search.fit(X_train_reduced, train_df['Tag Value'])

# Imprimir mejores resultados
print("Mejor configuración encontrada:")
print(grid_search.best_params_)
print(f"Mejor F1-macro: {grid_search.best_score_:.4f}")

print("\nResultados detallados por configuración:")
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
params = grid_search.cv_results_['params']
for mean, std, param in zip(means, stds, params):
    print(f"{param} -> F1-macro: {mean:.4f} (+/-{std:.4f})")

Fitting 5 folds for each of 54 candidates, totalling 270 fits
[CV] END learning_rate=0.01, max_depth=2, n_estimators=50, subsample=0.8; total time=   2.0s
[CV] END learning_rate=0.01, max_depth=2, n_estimators=50, subsample=0.8; total time=   2.2s
[CV] END learning_rate=0.01, max_depth=2, n_estimators=50, subsample=0.8; total time=   2.2s
[CV] END learning_rate=0.01, max_depth=2, n_estimators=50, subsample=0.8; total time=   2.1s
[CV] END learning_rate=0.01, max_depth=2, n_estimators=50, subsample=0.8; total time=   1.8s
[CV] END learning_rate=0.01, max_depth=2, n_estimators=50, subsample=1.0; total time=   2.3s
[CV] END learning_rate=0.01, max_depth=2, n_estimators=50, subsample=1.0; total time=   2.4s
[CV] END learning_rate=0.01, max_depth=2, n_estimators=50, subsample=1.0; total time=   2.3s
[CV] END learning_rate=0.01, max_depth=2, n_estimators=50, subsample=1.0; total time=   2.2s
[CV] END learning_rate=0.01, max_depth=2, n_estimators=50, subsample=1.0; total time=   2.4s
[CV] END

In [35]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score

# Definir el modelo de Gradient Boosting
gb_model = GradientBoostingClassifier(subsample=1.0,n_estimators=200, learning_rate=0.1, max_depth=5, random_state=0)

# Evaluar el modelo con validación cruzada usando las representaciones reducidas (X_train_reduced)
gb_scores = cross_val_score(gb_model, X_train_reduced, train_df['Tag Value'], cv=5, scoring='f1_macro')

# Imprimir los resultados
print("Gradient Boosting:")
print(f"F1-macro scores por fold: {gb_scores}")
print(f"F1-macro promedio: {gb_scores.mean():.4f}")

Gradient Boosting:
F1-macro scores por fold: [0.67592593 0.63653528 0.67474747 0.67921518 0.66369586]
F1-macro promedio: 0.6660


## Usando texto limpio

## Usando no-stopwrods